In [2]:
#Here we're going to polynomial expand the 4 most populated cols (the ones with)
#The fewest NaN entries, then recombine them with the 4 less populated cols.
#This captures their interaction terms.
#Would the result be similar if we had done fi / fj instead of fi*fj?

import numpy as np
import pandas as pd

#Create a dataframe and chop a triangular hole out of it.
X = pd.DataFrame(np.arange(80.).reshape(10,8))

for i in range(5,0,-1):
    for j in range(i, 6):
        X.set_value(i,j, np.nan)
X

0   1   2   3   4   5   6   7
0   0   1   2   3   4   5   6   7
1   8 NaN NaN NaN NaN NaN  14  15
2  16  17 NaN NaN NaN NaN  22  23
3  24  25  26 NaN NaN NaN  30  31
4  32  33  34  35 NaN NaN  38  39
5  40  41  42  43  44 NaN  46  47
6  48  49  50  51  52  53  54  55
7  56  57  58  59  60  61  62  63
8  64  65  66  67  68  69  70  71
9  72  73  74  75  76  77  78  79

In [4]:
#Reorder cols by number of non null entries
#How to do this pandorably? 
cols_counts = list((zip(X.columns.tolist(), X.count().tolist())))
most_pop_cols = [ elem[0] for elem in sorted(cols_counts, key=lambda cols_counts: cols_counts[1], reverse=True) ]
most_pop_cols

[0, 6, 7, 1, 2, 3, 4, 5]

In [5]:
#Mean impute missing data, need all float entries.
from sklearn.preprocessing import Imputer
imp = Imputer()
X = pd.DataFrame(imp.fit_transform(X))

In [6]:
#Slice out 4 cols with most entries
num_cols = 4
most, rest = most_pop_cols[0:num_cols], most_pop_cols[num_cols:]

#Polynomial expand most populated cols to get all interaction terms.
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2, interaction_only=True)
poly_most = pd.DataFrame(poly.fit_transform(X[most]))
poly_most

0   1   2   3          4     5     6            7     8            9   \
0   1   0   6   7   1.000000     0     0     0.000000    42     6.000000   
1   1   8  14  15  40.111111   112   120   320.888889   210   561.555556   
2   1  16  22  23  17.000000   352   368   272.000000   506   374.000000   
3   1  24  30  31  25.000000   720   744   600.000000   930   750.000000   
4   1  32  38  39  33.000000  1216  1248  1056.000000  1482  1254.000000   
5   1  40  46  47  41.000000  1840  1880  1640.000000  2162  1886.000000   
6   1  48  54  55  49.000000  2592  2640  2352.000000  2970  2646.000000   
7   1  56  62  63  57.000000  3472  3528  3192.000000  3906  3534.000000   
8   1  64  70  71  65.000000  4480  4544  4160.000000  4970  4550.000000   
9   1  72  78  79  73.000000  5616  5688  5256.000000  6162  5694.000000   

            10  
0     7.000000  
1   601.666667  
2   391.000000  
3   775.000000  
4  1287.000000  
5  1927.000000  
6  2695.000000  
7  3591.000000  
8  4615.000000  
9  5767.000000

In [8]:
#Concat poly expansion to unexpanded data
expanded_data = pd.concat([poly_most, X[rest]], axis=1) #join_axes=[df1.index])
expanded_data

0   1   2   3          4     5     6            7     8            9   \
0   1   0   6   7   1.000000     0     0     0.000000    42     6.000000   
1   1   8  14  15  40.111111   112   120   320.888889   210   561.555556   
2   1  16  22  23  17.000000   352   368   272.000000   506   374.000000   
3   1  24  30  31  25.000000   720   744   600.000000   930   750.000000   
4   1  32  38  39  33.000000  1216  1248  1056.000000  1482  1254.000000   
5   1  40  46  47  41.000000  1840  1880  1640.000000  2162  1886.000000   
6   1  48  54  55  49.000000  2592  2640  2352.000000  2970  2646.000000   
7   1  56  62  63  57.000000  3472  3528  3192.000000  3906  3534.000000   
8   1  64  70  71  65.000000  4480  4544  4160.000000  4970  4550.000000   
9   1  72  78  79  73.000000  5616  5688  5256.000000  6162  5694.000000   

            10  2          3          4   5   
0     7.000000   2   3.000000   4.000000   5  
1   601.666667  44  47.571429  50.666667  53  
2   391.000000  44  47.571429  50.666667  53  
3   775.000000  26  47.571429  50.666667  53  
4  1287.000000  34  35.000000  50.666667  53  
5  1927.000000  42  43.000000  44.000000  53  
6  2695.000000  50  51.000000  52.000000  53  
7  3591.000000  58  59.000000  60.000000  61  
8  4615.000000  66  67.000000  68.000000  69  
9  5767.000000  74  75.000000  76.000000  77

In [9]:
expanded_data.to_csv("poly_expanded.csv", index=False)